In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.pipeline import Pipeline
import joblib
import pickle

pd.pandas.set_option('display.max_columns', None)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving newProsperData.csv to newProsperData.csv


In [4]:
data = pd.read_csv("newProsperData.csv")

In [5]:
data.head()

,CreditGrade,LoanTenure,LoanStatus,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperRating (Alpha),ProsperScore,ListingCategory (numeric),BorrowerState,Occupation,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CurrentlyInGroup,CreditScoreRangeLower,CreditScoreRangeUpper,FirstRecordedCreditLine,CurrentCreditLines,OpenCreditLines,TotalCreditLinespast7years,OpenRevolvingAccounts,OpenRevolvingMonthlyPayment,InquiriesLast6Months,TotalInquiries,CurrentDelinquencies,AmountDelinquent,DelinquenciesLast7Years,PublicRecordsLast10Years,PublicRecordsLast12Months,RevolvingCreditBalance,BankcardUtilization,AvailableBankcardCredit,TotalTrades,TradesNeverDelinquent (percentage),TradesOpenedLast6Months,DebtToIncomeRatio,IncomeRange,IncomeVerifiable,StatedMonthlyIncome,LoanCurrentDaysDelinquent,LoanMonthsSinceOrigination,LoanOriginalAmount,LoanOriginationQuarter,MonthlyLoanPayment,LP_CustomerPayments,LP_CustomerPrincipalPayments,LP_InterestandFees,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors,EMI,ELA,InterestAmount,TotalAmount,ROI,PROI
0,3,36,2,0.16516,0.1580,0.1380,0.16150,0.0724,0.09170,4.0,7,6.0,0,5,36,8,2.0,1,1,640.0,659.0,8637,5.0,4.0,12.0,1,24.0,3.0,3.0,2.0,472.0,4.0,0.0,0.0,0.0,0.00,1500.0,11.0,0.81,0.0,0.17,3,1,3083.333333,0,78,9425,17,330.43,11396.14,9425.00,1971.14,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258,1496.764570,63034.0,1489.150,10914.150,0.136442,0.105405
1,7,36,3,0.12016,0.0920,0.0820,0.07960,0.0249,0.05470,6.0,0,7.0,2,5,42,0,44.0,0,0,680.0,699.0,6615,14.0,14.0,29.0,13,389.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,3989.0,0.21,10266.0,29.0,1.00,2.0,0.18,4,1,6125.000000,0,0,10000,7,318.93,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1,0.000000,43120.0,920.000,10920.000,0.084249,0.205405
2,6,36,2,0.28269,0.2750,0.2400,0.16150,0.0724,0.09170,4.0,7,6.0,0,10,36,3,67.0,0,1,480.0,499.0,8925,10.0,9.0,3.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,8549.0,0.60,4100.0,22.0,0.94,0.0,0.06,6,1,2083.333333,0,86,3001,1,123.32,4186.63,3001.00,1185.63,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41,825.406311,32711.0,825.275,3826.275,0.215686,0.105405
3,7,36,3,0.12528,0.0974,0.0874,0.08490,0.0249,0.06000,6.0,0,9.0,16,10,51,0,113.0,1,0,800.0,819.0,2245,5.0,5.0,29.0,7,115.0,0.0,1.0,4.0,10056.0,14.0,0.0,0.0,1444.0,0.04,30754.0,26.0,0.76,0.0,0.15,3,1,2875.000000,0,16,10000,31,321.45,5143.20,4091.09,1052.11,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158,413.021192,45064.0,974.000,10974.000,0.088755,0.105405
4,7,36,3,0.24614,0.2085,0.1985,0.18316,0.0925,0.09066,3.0,4,4.0,2,23,19,0,44.0,1,0,680.0,699.0,9496,19.0,19.0,49.0,6,220.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,6193.0,0.81,695.0,39.0,0.95,2.0,0.26,2,1,9583.333333,0,6,15000,23,563.97,2819.85,1563.22,1256.63,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20,326.288414,127590.0,3127.500,18127.500,0.172528,0.205405


In [6]:
data.shape

(113929, 68)

In [7]:
data.isnull().sum().sum()

0

In [8]:
# The LoanStatus column is already encoded. But for the pipeline, the LoanStatus should be in the binary format.

data['LoanStatus']

,LoanStatus
0,2
1,3
2,2
3,3
4,3
...,...
113924,3
113925,5
113926,3
113927,2


In [9]:
# This dataset contains LoanStatus with original values.

data_loanStatus = pd.read_csv("LoanStatus.csv")

In [10]:
data_loanStatus.head()

,LoanStatus
0,Completed
1,Current
2,Completed
3,Current
4,Current


In [11]:
data_loanStatus.shape

(113929, 1)

In [12]:
# Replacing the encoded LoanStatus column into its original values.

data['LoanStatus'] = data_loanStatus['LoanStatus'].copy()

In [13]:
data.head()

,CreditGrade,LoanTenure,LoanStatus,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperRating (Alpha),ProsperScore,ListingCategory (numeric),BorrowerState,Occupation,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CurrentlyInGroup,CreditScoreRangeLower,CreditScoreRangeUpper,FirstRecordedCreditLine,CurrentCreditLines,OpenCreditLines,TotalCreditLinespast7years,OpenRevolvingAccounts,OpenRevolvingMonthlyPayment,InquiriesLast6Months,TotalInquiries,CurrentDelinquencies,AmountDelinquent,DelinquenciesLast7Years,PublicRecordsLast10Years,PublicRecordsLast12Months,RevolvingCreditBalance,BankcardUtilization,AvailableBankcardCredit,TotalTrades,TradesNeverDelinquent (percentage),TradesOpenedLast6Months,DebtToIncomeRatio,IncomeRange,IncomeVerifiable,StatedMonthlyIncome,LoanCurrentDaysDelinquent,LoanMonthsSinceOrigination,LoanOriginalAmount,LoanOriginationQuarter,MonthlyLoanPayment,LP_CustomerPayments,LP_CustomerPrincipalPayments,LP_InterestandFees,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors,EMI,ELA,InterestAmount,TotalAmount,ROI,PROI
0,3,36,Completed,0.16516,0.1580,0.1380,0.16150,0.0724,0.09170,4.0,7,6.0,0,5,36,8,2.0,1,1,640.0,659.0,8637,5.0,4.0,12.0,1,24.0,3.0,3.0,2.0,472.0,4.0,0.0,0.0,0.0,0.00,1500.0,11.0,0.81,0.0,0.17,3,1,3083.333333,0,78,9425,17,330.43,11396.14,9425.00,1971.14,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258,1496.764570,63034.0,1489.150,10914.150,0.136442,0.105405
1,7,36,Current,0.12016,0.0920,0.0820,0.07960,0.0249,0.05470,6.0,0,7.0,2,5,42,0,44.0,0,0,680.0,699.0,6615,14.0,14.0,29.0,13,389.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,3989.0,0.21,10266.0,29.0,1.00,2.0,0.18,4,1,6125.000000,0,0,10000,7,318.93,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1,0.000000,43120.0,920.000,10920.000,0.084249,0.205405
2,6,36,Completed,0.28269,0.2750,0.2400,0.16150,0.0724,0.09170,4.0,7,6.0,0,10,36,3,67.0,0,1,480.0,499.0,8925,10.0,9.0,3.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,8549.0,0.60,4100.0,22.0,0.94,0.0,0.06,6,1,2083.333333,0,86,3001,1,123.32,4186.63,3001.00,1185.63,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41,825.406311,32711.0,825.275,3826.275,0.215686,0.105405
3,7,36,Current,0.12528,0.0974,0.0874,0.08490,0.0249,0.06000,6.0,0,9.0,16,10,51,0,113.0,1,0,800.0,819.0,2245,5.0,5.0,29.0,7,115.0,0.0,1.0,4.0,10056.0,14.0,0.0,0.0,1444.0,0.04,30754.0,26.0,0.76,0.0,0.15,3,1,2875.000000,0,16,10000,31,321.45,5143.20,4091.09,1052.11,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158,413.021192,45064.0,974.000,10974.000,0.088755,0.105405
4,7,36,Current,0.24614,0.2085,0.1985,0.18316,0.0925,0.09066,3.0,4,4.0,2,23,19,0,44.0,1,0,680.0,699.0,9496,19.0,19.0,49.0,6,220.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,6193.0,0.81,695.0,39.0,0.95,2.0,0.26,2,1,9583.333333,0,6,15000,23,563.97,2819.85,1563.22,1256.63,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20,326.288414,127590.0,3127.500,18127.500,0.172528,0.205405


In [14]:
# Converting the LoanStatus column into "defaulted" and "not_defaulted"

def classify_loan(row):
    # If LoanStatus is already "Defaulted" or "Chargedoff", mark as Defaulted
    if row['LoanStatus'] in ['Defaulted', 'Chargedoff']:
        return 1

    # If LoanCurrentDaysDelinquent exceeds 90 days and the loan is closed (has a ClosedDate), mark as Defaulted
    elif row['LoanCurrentDaysDelinquent'] > 180 and pd.notnull(row['ClosedDate']):
        return 1

    # Otherwise, consider the loan Not Defaulted
    else:
        return 0

In [15]:
# Apply the classification logic

data['IsDefaulted'] = data.apply(classify_loan, axis=1)

In [16]:
data['IsDefaulted'].value_counts()

,count
IsDefaulted,
0,96920
1,17009


In [17]:
# Now remove LoanStatus column.

data = data.drop(['LoanStatus'], axis=1)

In [18]:
data.head()

,CreditGrade,LoanTenure,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperRating (Alpha),ProsperScore,ListingCategory (numeric),BorrowerState,Occupation,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CurrentlyInGroup,CreditScoreRangeLower,CreditScoreRangeUpper,FirstRecordedCreditLine,CurrentCreditLines,OpenCreditLines,TotalCreditLinespast7years,OpenRevolvingAccounts,OpenRevolvingMonthlyPayment,InquiriesLast6Months,TotalInquiries,CurrentDelinquencies,AmountDelinquent,DelinquenciesLast7Years,PublicRecordsLast10Years,PublicRecordsLast12Months,RevolvingCreditBalance,BankcardUtilization,AvailableBankcardCredit,TotalTrades,TradesNeverDelinquent (percentage),TradesOpenedLast6Months,DebtToIncomeRatio,IncomeRange,IncomeVerifiable,StatedMonthlyIncome,LoanCurrentDaysDelinquent,LoanMonthsSinceOrigination,LoanOriginalAmount,LoanOriginationQuarter,MonthlyLoanPayment,LP_CustomerPayments,LP_CustomerPrincipalPayments,LP_InterestandFees,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors,EMI,ELA,InterestAmount,TotalAmount,ROI,PROI,IsDefaulted
0,3,36,0.16516,0.1580,0.1380,0.16150,0.0724,0.09170,4.0,7,6.0,0,5,36,8,2.0,1,1,640.0,659.0,8637,5.0,4.0,12.0,1,24.0,3.0,3.0,2.0,472.0,4.0,0.0,0.0,0.0,0.00,1500.0,11.0,0.81,0.0,0.17,3,1,3083.333333,0,78,9425,17,330.43,11396.14,9425.00,1971.14,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258,1496.764570,63034.0,1489.150,10914.150,0.136442,0.105405,0
1,7,36,0.12016,0.0920,0.0820,0.07960,0.0249,0.05470,6.0,0,7.0,2,5,42,0,44.0,0,0,680.0,699.0,6615,14.0,14.0,29.0,13,389.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,3989.0,0.21,10266.0,29.0,1.00,2.0,0.18,4,1,6125.000000,0,0,10000,7,318.93,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1,0.000000,43120.0,920.000,10920.000,0.084249,0.205405,0
2,6,36,0.28269,0.2750,0.2400,0.16150,0.0724,0.09170,4.0,7,6.0,0,10,36,3,67.0,0,1,480.0,499.0,8925,10.0,9.0,3.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,8549.0,0.60,4100.0,22.0,0.94,0.0,0.06,6,1,2083.333333,0,86,3001,1,123.32,4186.63,3001.00,1185.63,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41,825.406311,32711.0,825.275,3826.275,0.215686,0.105405,0
3,7,36,0.12528,0.0974,0.0874,0.08490,0.0249,0.06000,6.0,0,9.0,16,10,51,0,113.0,1,0,800.0,819.0,2245,5.0,5.0,29.0,7,115.0,0.0,1.0,4.0,10056.0,14.0,0.0,0.0,1444.0,0.04,30754.0,26.0,0.76,0.0,0.15,3,1,2875.000000,0,16,10000,31,321.45,5143.20,4091.09,1052.11,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158,413.021192,45064.0,974.000,10974.000,0.088755,0.105405,0
4,7,36,0.24614,0.2085,0.1985,0.18316,0.0925,0.09066,3.0,4,4.0,2,23,19,0,44.0,1,0,680.0,699.0,9496,19.0,19.0,49.0,6,220.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,6193.0,0.81,695.0,39.0,0.95,2.0,0.26,2,1,9583.333333,0,6,15000,23,563.97,2819.85,1563.22,1256.63,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20,326.288414,127590.0,3127.500,18127.500,0.172528,0.205405,0


In [19]:
# Split data into features and target

X = data.drop(['IsDefaulted', 'EMI', 'ELA', 'PROI'], axis=1)
y_class = data['IsDefaulted']
y_reg = data[['EMI', 'ELA', 'PROI']]

# Split into training and testing sets
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X, y_class, y_reg, test_size=0.2, random_state=42
)

**Load the Classifier and Regressor models that's been saved earlier and define the number of features**

In [20]:
# Load the classifier model

clf_model = files.upload()

Saving model.sav to model.sav


In [21]:
# Load the regressor model

reg_model = files.upload()

Saving best_linear_regression_model.pkl to best_linear_regression_model.pkl


**Deserialize the uploaded models**

In [22]:
# Classifier model
clf_model_filename = list(clf_model.keys())[0]  # Get the filename
clf_model = joblib.load(clf_model_filename)  # Replace with pickle.load() if it’s a .pkl file

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [23]:
# Regressor model
reg_model_filename = list(reg_model.keys())[0]

# Load the model using joblib
reg_model = joblib.load(reg_model_filename)

In [24]:
# Ensure clf_model and reg_model are valid model objects
print(f"Classifier type: {type(clf_model)}")
print(f"Regressor type: {type(reg_model)}")

Classifier type: <class 'lightgbm.sklearn.LGBMClassifier'>
Regressor type: <class 'sklearn.pipeline.Pipeline'>


In [25]:
num_features = 10

# Classification Model

1. **Train Classification Model**: Train a binary classification model on the training data.
2. **Make Predictions**: Use the classification model to make predictions on the test data.

In [26]:
clf_model.fit(X_train, y_class_train)

# Make predictions
y_class_pred = clf_model.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 13668, number of negative: 77475
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049689 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8924
[LightGBM] [Info] Number of data points in the train set: 91143, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

# Regression Model

1. **Filter Data Based on Classification**: Use the predictions from the classification model to filter data for the regression model.
2. **Train Regression Model**: Train a regression model on the filtered data.

In [27]:
# Filter training data based on classification predictions
X_train_reg = X_train[y_class_train == 1]
y_reg_train_filtered = y_reg_train[y_class_train == 1]

# Filter test data based on classification predictions
X_test_reg = X_test[y_class_pred == 1]
y_reg_test_filtered = y_reg_test[y_class_pred == 1]

# Initialize and train the regression model
reg_model.fit(X_train_reg, y_reg_train_filtered)

# Make predictions
y_reg_pred = reg_model.predict(X_test_reg)

# Evaluation

1. **Evaluate Classification Model**: Evaluate the performance of the classification model using metrics like accuracy, precision, recall, and F1-score.
2. **Evaluate Regression Model**: Evaluate the performance of the regression model using metrics like Mean Absolute Error (MAE), Mean Squared Error (MSE), or R-squared.


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [30]:
# Evaluate classification model
accuracy = accuracy_score(y_class_test, y_class_pred)
precision = precision_score(y_class_test, y_class_pred)
recall = recall_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred)

print(f"Classification Model - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

Classification Model - Accuracy: 0.9996927938207671, Precision: 0.9991017964071857, Recall: 0.9988027536665669, F1-Score: 0.9989522526567879


In [31]:
# Evaluate regression model
mae = mean_absolute_error(y_reg_test_filtered, y_reg_pred)
mse = mean_squared_error(y_reg_test_filtered, y_reg_pred)
r2 = r2_score(y_reg_test_filtered, y_reg_pred)

print(f"Regression Model - MAE: {mae}, MSE: {mse}, R2: {r2}")

Regression Model - MAE: 1856.4763949551664, MSE: 60936751.121367484, R2: 0.788905084705886


**Feature Selection Using Mutual Info for both classifier and regressor models**

In [32]:
from sklearn.feature_selection import _mutual_info
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, r2_score

### Full Pipeline

Integrate the classification and regression models into a single pipeline using a custom pipeline object or using a library like `scikit-learn`'s `Pipeline`.

In [33]:
class StaticScoreFunction:
    """
    Custom score function for SelectKBest that uses precomputed scores.
    """
    def __init__(self, combined_scores):
        self.combined_scores = combined_scores

    def __call__(self, X, y):
        # The function ignores `X` and `y` and returns the precomputed scores
        return self.combined_scores

In [34]:
class CustomPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, clf, reg, num_features):
        self.clf = clf
        self.reg = reg
        self.num_features = num_features

    def fit(self, X, y_class, y_reg):
        # Calculate mutual information for both tasks
        mi_reg = mutual_info_regression(X, y_reg.mean(axis=1))
        mi_class = mutual_info_classif(X, y_class)

        # Combine scores
        combined_scores = (mi_reg + mi_class) / 2

        # Use the global scoring class with precomputed scores
        score_func = StaticScoreFunction(combined_scores)

        # Select top features using the static scoring function
        self.selector = SelectKBest(score_func=score_func, k=self.num_features)
        #self.selector.fit(X, y_reg.mean(axis=1))
        self.selector.fit(X, y_class)  # Fit using classification labels
        X_selected = self.selector.transform(X)

        # Fit the classifier and regressor
        self.clf.fit(X_selected, y_class)

        X_filtered = X_selected[y_class == 1]
        y_reg_filtered = y_reg[y_class == 1]
        self.reg.fit(X_filtered, y_reg_filtered)
        return self

    def transform(self, X):
        # Transform the input data using the fitted selector
        return self.selector.transform(X)

    def predict(self, X):
        # Select features using the same selector
        X_selected = self.transform(X)

        # Make predictions for both tasks
        y_class_pred = self.clf.predict(X_selected)

        X_filtered = X_selected[y_class_pred == 1]
        y_reg_pred = self.reg.predict(X_filtered)
        return y_class_pred, y_reg_pred

# Create an instance of the CustomPipeline
pipeline = CustomPipeline(clf=clf_model, reg=reg_model, num_features=num_features)

# Fit the pipeline
pipeline.fit(X_train, y_class_train, y_reg_train)

# Make predictions
y_class_pred, y_reg_pred = pipeline.predict(X_test)

[LightGBM] [Info] Number of positive: 13668, number of negative: 77475
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009682 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2534
[LightGBM] [Info] Number of data points in the train set: 91143, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


**Custom Pipeline Evaluation**

In [35]:
# Classification metrics
print("Classification Metrics:")
print(f"Accuracy: {accuracy_score(y_class_test, y_class_pred):.4f}")
print(f"Precision: {precision_score(y_class_test, y_class_pred, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_class_test, y_class_pred, average='weighted'):.4f}")
print(f"F1 Score: {f1_score(y_class_test, y_class_pred, average='weighted'):.4f}")

print("\n")

# Regression metrics for each target
print("Regression Metrics:")

# Filter y_reg_test based on classification predictions
y_reg_test_filtered = y_reg_test[y_class_pred == 1].reset_index(drop=True)

for i, target in enumerate(y_reg_test_filtered.columns):
    print(f"\nMetrics for {target}:")
    print(f"MAE: {mean_absolute_error(y_reg_test_filtered.iloc[:, i], y_reg_pred[:, i]):.4f}")
    print(f"MSE: {mean_squared_error(y_reg_test_filtered.iloc[:, i], y_reg_pred[:, i]):.4f}")
    print(f"R^2: {r2_score(y_reg_test_filtered.iloc[:, i], y_reg_pred[:, i]):.4f}")

Classification Metrics:
Accuracy: 0.9856
Precision: 0.9856
Recall: 0.9856
F1 Score: 0.9856


Regression Metrics:

Metrics for EMI:
MAE: 0.7434
MSE: 2.7193
R^2: 1.0000

Metrics for ELA:
MAE: 5746.7213
MSE: 385309879.9787
R^2: 0.8590

Metrics for PROI:
MAE: 0.0274
MSE: 0.0011
R^2: 0.5969


In [36]:
# Save the pipeline

joblib.dump(pipeline, 'combined_pipeline.pkl')

['combined_pipeline.pkl']